Summary: Setup tensorflow with phone brand feature(onehot), using stochastic gradient descent

In [1]:
import numpy as np
import pandas as pd

gender_age_test = pd.read_csv('./input/gender_age_test.csv', index_col='device_id')
gender_age_train = pd.read_csv('./input/gender_age_train.csv', index_col='device_id')
phone_brand_device_model = pd.read_csv('./input/phone_brand_device_model.csv')
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id', keep='first').set_index('device_id')

In [2]:
phone_train = gender_age_train.merge(phone_brand_device_model, how='left', left_index=True, right_index=True)

In [3]:
brand_onehot = pd.get_dummies(phone_train.phone_brand)
model_onehot = pd.get_dummies(phone_train.device_model)
group_onehot = pd.get_dummies(phone_train.group)

In [4]:
phone_train = phone_train.merge(brand_onehot, how='left', left_index=True, right_index=True)
phone_train = phone_train.merge(model_onehot, how='left', left_index=True, right_index=True)
phone_train = phone_train.merge(group_onehot, how='left', left_index=True, right_index=True)

In [5]:
phone_train.head()

,gender,age,group,phone_brand,device_model,E派,HTC,LG,LOGO,Lovme,...,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,,,,,,,,,,
-8076087639492063270,M,35,M32-38,小米,MI 2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
-2897161552818060146,M,35,M32-38,小米,MI 2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
-8260683887967679142,M,35,M32-38,小米,MI 2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
-4938849341048082022,M,30,M29-31,小米,红米note,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
245133531816851882,M,30,M29-31,小米,MI 3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
result_list = ['M32-38', 'M29-31', 'F24-26', 'F33-42', 'F27-28', 'M39+', 'M23-26', \
               'M27-28', 'M22-', 'F43+', 'F23-', 'F29-32']

In [7]:
feature_list = list(set(phone_train.columns.values.tolist()) - \
                    set(['gender','age','group','phone_brand','device_model','groupencode']) - \
                    set(result_list))

In [8]:
currentCount = 0
def get_next_batch(count):
    global currentCount
    train_data = phone_train[currentCount:currentCount+count][feature_list].values
    train_result = phone_train[currentCount:currentCount+count][result_list].values
    currentCount += count
    return (train_data, train_result)

In [9]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [10]:
x = tf.placeholder(tf.float32, shape=[None, 1558])
y_ = tf.placeholder(tf.float32, shape=[None, 12])

In [11]:
W = tf.Variable(tf.zeros([1558,12]))
b = tf.Variable(tf.zeros([12]))

In [12]:
sess.run(tf.initialize_all_variables())

In [13]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [14]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [16]:
for i in range(1000):
    batch = get_next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [17]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [18]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))